In [1]:
import pandas as pd
import numpy as np
from dateutil.parser import parse
from datetime import datetime, timedelta
import math

samplesize = 50000

In [ ]:




def _gender(data):
    try:
        if data and data.lower()=='male': return 1
        else: return 0
    except: return -1
    
def _age(data):
    try:
        if data and str(data) !='nan':
            bd = parse(data)
            diff = datetime.now() - bd
            return diff.days 
        else: return 0
    except Exception as e: 
        print (e)
        print (data)
        return -1
    
def _nationality(data):
    if data and 'singapore' in data.lower(): return 1   
    else: return 0
    
def _resi_status(data):
    if data and  data.lower() == 'citizen' or  data.lower() == 'pr': return 1   
    else: return 0
    
def _yesno(data):
    if data:
        if str(data).lower() == 'yes' : return 1 
        else: return 0
    else: return -1
    
def _occupation(data):
    if data and 'tour' in str(data).lower(): return 1   
    else: return 0
    
def _self_employ_time(data):
    import calendar
    try:
        if data and str(data) != 'nan':
            
            
            _m = list(calendar.month_abbr).index(data.split(";")[0]) 

            

            __y = data.split(";")[1].replace('Before','').replace(' ','') # get rid of silly "Before"
            _y = int(__y)

            #print ( (datetime.now() -   datetime(_y,_m,1)).days)
            return (datetime.now() -   datetime(_y,_m,1)).days
            #else: 
            #    ym = data.split(';')[2]
            #    _y = int(ym.split("-")[0])
            #    _m = int(ym.split("-")[1])
            #    return (datetime.now() -   datetime(_y,_m,1)).days
                
        else: return 0
    except Exception as e: 
        print ('mthyear exception')
        print (data)
        print (e)
        return 0
    
def _log(data):
    try:
        #print (para)
        if data and float(data)>0:return math.log10(float(data))
        else: return 0
    except Exception as e: 
        #print (e)
        return -1
def _int(data):
    if data and str(data) != 'nan': return int(data)
    else: return 0
    
    
from textblob import TextBlob #sentiment analysis
import nltk
nltk.download('punkt')

import statistics
def _sentiment_of(text):
    if text:
        blob = TextBlob(str(text))
        _sentiments = []
        for sentence in blob.sentences:
            _sentiments.append(sentence.sentiment.polarity)
        return  statistics.mean(_sentiments) 
    else: return 0
    
def _interested(data):
    if data and 'contact' in str(data): return 1
    else: return 0
    
def _label_outcome(data):
    if data:
        if str(data) == 'Approve - 1000': return 1000
        elif str(data) == 'Approve - 800': return 800
        elif str(data) == 'Approve - 400': return 400
        else: return 0
    else: return -1

In [ ]:


df = pd.read_csv('output2.csv', dtype='unicode') # this is the final file!


#_result = df[0:samplesize]
_result = df
print ('start')
old_columns = [i for i in _result.columns]

_result['_gender']=_result['Gender'].apply(_gender)
_result['_age']=_result['Date of Birth'].apply(_age)
_result['_nationality']=_result['Nationality'].apply(_nationality)
_result['_resi_status']=_result['Residential Status'].apply(_resi_status)

_result = pd.concat([_result,pd.get_dummies(_result['Housing Type'], prefix='onehot')],axis=1)
_result['_hdb_registered_address']=_result['Registered Address'].apply(_yesno)
_result = pd.concat([_result,pd.get_dummies(_result['Highest Education'], prefix='onehot')],axis=1)
_result['_occupation']=_result['Occupation'].apply(_occupation)
#_result['_av']=_result['Annual Value (AV) of  registered address'].apply(_av,args=(-1,))
_result['_av_value']=_result['Annual Value (AV) of  registered address'].apply(_log)

_result['__temp']=_result['Self Employed Since (Month)']+';'+_result['Self Employed Since (Year)']
_result['_self_employ_time']=_result['__temp'].apply(_self_employ_time)

print ('50%') 
_result['_2018_t']=_result['Income in 2018 (NOA YA2019) - Trade income'].apply(_log)
_result['_2018_e']=_result['\xa0Income in 2018 (NOA YA2019) - Employment income'].apply(_log)
_result['_2019_e']=_result['Income in 2019 (NOA YA2020) - Trade income'].apply(_log)
_result['_2019_t']=_result['Income in 2019 (NOA YA2020) - Employment income'].apply(_log)
_result['_2020_e']=_result['Income in 2020 (Jan-Mar) - Trade income'].apply(_log)
_result['_2020_t']=_result['Income in 2020 (Jan-Mar) - Employment income'].apply(_log)
_result['s_2019_i']=_result['Spouse\'s Income in 2019'].apply(_log)
_result['s_2020_i']=_result['Spouse\'s Income in 2020 (Jan-Mar)'].apply(_log)

_result['_child']=_result['No. of Singaporean Children < 21'].apply(_int)
_result['_parent']=_result['No. of Singaporean Parents > 64'].apply(_int)

_result = pd.concat([_result,pd.get_dummies(_result['Marital Status'], prefix='onehot')],axis=1)
_result['_married']=_result['Marriage Date'].apply(_age)
_result['_divorced']=_result['Divorce Date'].apply(_age)

_result['_hdb']=_result['Number of HDB properties own'].apply(_int)
_result['_pte']=_result['Number of private properties own'].apply(_int)
_result['_comz']=_result['Number of commercial properties own'].apply(_int)
_result['_comz_av']=_result['Annual Value (AV) of commercial properties own'].apply(_log)
_result['_comz_operate']=_result['Biz Operations from  commercial properties'].apply(_yesno)


_result['_hardship_sentiment']=_result['Please describe hardship (if any)'].apply(_sentiment_of)
_result['_ntuc_covid']=_result['Interested in NTUC Covid19 Programme'].apply(_interested)

_result['LABEL']=_result['Final Outcome Combined'].apply(_label_outcome)


_result = _result.drop(old_columns,axis=1)
print ('end')

_result.to_csv ('_final.csv', index = False, header=True)

In [ ]:
from machine import Machine

In [ ]:
m = Machine()


In [6]:
m.train()

Train on 40000 samples, validate on 10000 samples
Epoch 1/150
40000/40000 [==============================] - 5s 136us/sample - loss: 22.0500 - accuracy: 0.5324 - val_loss: 4.0148 - val_accuracy: 0.5589
Epoch 2/150
40000/40000 [==============================] - 3s 67us/sample - loss: 4.1619 - accuracy: 0.5623 - val_loss: 2.1797 - val_accuracy: 0.6124
Epoch 3/150
40000/40000 [==============================] - 3s 69us/sample - loss: 4.2126 - accuracy: 0.5741 - val_loss: 3.7339 - val_accuracy: 0.4742
Epoch 4/150
40000/40000 [==============================] - 3s 69us/sample - loss: 3.7202 - accuracy: 0.5807 - val_loss: 2.9225 - val_accuracy: 0.5895
Epoch 5/150
40000/40000 [==============================] - 3s 68us/sample - loss: 3.5853 - accuracy: 0.5833 - val_loss: 2.3797 - val_accuracy: 0.6672
Epoch 6/150
40000/40000 [==============================] - 3s 70us/sample - loss: 3.7852 - accuracy: 0.5783 - val_loss: 2.8385 - val_accuracy: 0.5989
Epoch 7/150
40000/40000 [=======================

In [7]:
import tensorflow as tf
load_model = tf.keras.models.load_model
model = load_model('_modelv2.h5',custom_objects={'tf': tf}, compile=False)


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 15)                735       
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 64        
Total params: 799
Trainable params: 799
Non-trainable params: 0
_________________________________________________________________


In [11]:
import numpy as np
#input = [0,11932,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.0,1542,3.3010299956639813,0.0,3.681241237375587,0.0,3.462397997898956,0.0,5.012837224705172,4.0484029561527395,2,0,0,1,0,0,2710,0,1,0,0,0.0,0,0.0,0] #1lk
input = [1,10224,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.0,1177,4.792454727670386,4.091491094267951,4.685222065334621,2.552668216112193,3.9030899869919438,0.0,0.0,0.0,0,2,0,0,1,0,0,0,1,0,0,0.0,0,0.0,1] #1000
#input = [1,23779,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4.346352974450639,10491,0.0,0.0,4.013427127070696,0.0,0.0,0.0,4.621124329567078,4.0092383709684665,0,0,0,1,0,0,13043,0,0,1,0,0.0,0,0.0,0] #800
#input = [1,13678,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,386,0.0,0.0,3.97799780995874,4.17897694729317,3.547528576459782,0.0,4.89368965098515,4.354146870586911,1,0,0,1,0,0,2171,0,1,0,0,0.0,0,-0.07583333333333332,1] #800
#input = [1,13336,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,537,0.0,0.0,4.452430493699599,4.4123261221462435,0.0,0.0,4.655503396249793,3.3820170425748683,1,0,0,1,0,0,2500,0,1,0,0,0.0,0,0.03333333333333333,1] #1k
#input = [0,17792,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,4.357934847000454,4920,4.556302500767287,0.0,4.477121254719663,0.0,3.8750612633917,0.0,4.832508912706237,4.230448921378274,2,2,0,1,0,0,8657,0,0,1,0,0.0,0,0.0,0] #800

#input = [0, 20822, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3.8785217955012063, 0, 0, 0, 4.109747237713228, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, -1, -99, 0]
#input = [1, 20880, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 902, 3.964118143151485, 3.964118143151485, 3.2593549273080344, 3.2593549273080344, 3.3010299956639813, 3.3010299956639813, 3.964118143151485, 3.9242792860618816, 0, 0, 0, 1, 0, 0, 11275, 0, 1, 0, 0, 0, -1, 0.004444444444444445, 1]

_i = np.reshape(input,(len(input),1))

_a = model.predict([_i.T])
print (_a)
np.argmax(_a)  #0 = 0,1=400, 2=800, 3 = 1k

[[2.5097039e-01 1.4536532e-04 1.6739985e-02 7.3214418e-01]]


3

In [21]:
m.y[0:15]

array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.]], dtype=float32)